# scheduler 源码

## SchedulerMinHeap.js

是一个最小堆的序列

从存储结构看，最小堆是一个数组。
从逻辑结构看，最小堆是一棵完全二叉树

完全二叉树：
`A Complete Binary Tree （CBT) is a binary tree in which every level, 
except possibly the last, is completely filled, and all nodes 
are as far left as possible.`

形如：
```
     o
    / \
   o   o
  / \ / 
 o  oo   
```

In [ ]:
{
    type Node = {
        id:number,
        sortIndex:number
    }
    type Heap = Array<Node>;

    function push(heap:Heap,node:Node):void {
        const index = head.length;
        head.push(node);
        siftUp(head,node,index);
    }

    function peek(heaqp:Heap):Node|null {
        const first = head[0];
        return first === undefined?null:first;
    }

    function pop(heap:Heap):Node|null {
        const first = head[0];
        if(first!==undefined){
            const last = heap.pop();
            if(last !== first) {
                heap[0] = last;
                siftDown(heap,last,0);
            }
            return first;
        }else {
            return null;
        }
    }

    function siftUp(heap,node,i):void {
        let index = i;
        while(true){
            const parentIndex = Math.floor((index-1)/2);
            const parent = heap[parentIndex];
            if(parent !== undefined && compare(parent,node)>0){
                heap[parentIndex] = node;
                heap[index] = parent;
                index = parentIndex;
            }else{
                return;
            }
        }
    }

    function siftDown(heap,node,i):void {
        let index = i;
        const length = heap.length;
        while(index < length){
            const leftIndex = (index+1)*2-1;
            const left = heap[leftIndex];
            const rightIndex = leftIndex+1;
            const right = heap[rightIndex];

            if(left !== undefined && compare(left,node)<0){
                if(right !== undefined && compare(right,left)<0){
                    heap[index] = right;
                    heap[rightIndex] = node;
                    index = rightIndex;
                }else{
                    heap[index] = left;
                    heap[leftIndex] = node;
                    index = leftIndex;
                }
            }else if(right !== undefine && compare(right,node)<0) {
                heap[index] = right;
                heap[rightIndex] = node;
                index = rightIndex;
            }else {
                return;
            }
        }
    }

    function compare(a:Node,b:Node):boolean {
        const diff = a.sortIndex - b.sortIndex;
        return diff !== 0? diff:a.id - b.id;
    }
}

think list

- [x] Node 结构为什么要是 id sortIndex 
  - [x] id ans: id 是 taskCounter 是一个自增变量
  - [x] sortIndex ans: sortIndex 默认 -1  当逻辑进行中 可能是startTime 或 expirationTime  这个是用于执行优先级的一个参数 优先级最后也会换算成时间
- [x] push 为什么用siftUp ans: 二叉堆的性质 添加后要向上浮动
- [x] pop 为什么用siftDown ans: 二叉堆的性质 弹出后要向下重新排列
- [x] 最小/大堆
  - [x] 性质 ans: 1. 完全二叉树（使用完全二叉树是为了容易存储 节点i的左子节点 i*2 右子节点 i*2+1） 2. 每个节点值都小于或等于它的子节点。
  - [x] 操作 ans: 
              插入 放到最后然后与父节点对比 大于父节点 停止比较， 小于父节点 与父节点交换位置，继续上述比较。
              弹出 取出最后一个子节点，当作新的根节点，然后与子节点比较，与最小的子节点更换位置，知道其大于所有子节点。

## SchedulePriorities.js

任务优先级

In [ ]:
{
    type PriorityLeavel = 0|1|2|3|4|5;

    const NoPriority = 0;
    const ImmediatePriority = 1;
    const UserBlockingPriority = 2;
    const NormalPriority = 3;
    const LowPriority = 4;
    const IdlePriority = 5;
}

注释上说想用 `symbols` 但这样是否就不好去比较优先级了。 (ans: 并不会 这里的值其实只是在case 中使用 最终还是会换算成消耗时间加入到具体的task的sortIndex上)

## SchedulerHostConfig.js

. requestHostCallback
. requestHostTimeout
. cancelHostTimeout
. shouldYieldToHost
. getCurrentTime
. forceFrameRate
. requestRaint

具体的实现方法在 `forks/SchedulerHostConfig.defualt.js`里。

### SchedulerHostConfig.default.js

the dom scheduler implementation is similar to requestIdleCallback.

it worsk by scheduling a requestAnimationFrame, storing the time for the start of the frame,
then scheduling a postMessage which gets scheduled after paint.

within the postMessage handler do as much work as possible until time + frame rate.

by separating the idle call into a separate event tick we ensure that layout, paint and other browser work is counted against the available time.

the frame rate is dynamically adjusted.

In [ ]:
// 判断是否为dom 环境

typeof window === 'undefined' || typeof MessageChannel !== 'function'

之所以会用`messagechannel`判断 是因为用到了 `messagechannel`

In [ ]:
{
    //非dom环境下的实现方法。
    let _callback:null|any = null;
    let _timeoutID:null|number = null;
    
    const initialTime:number = Date.now();
    const getCurrentTime = function():number{
     return Date.now() - initialTime;
    };

    const _flushCallback = function():void{
        if(_callback !==null ){
            try {
                const currentTime:number = getCurrentTime();
                const hasRemainingTime:boolean = true;
                _callback(hasRemainingTime,currentTime);
                _callback = null;
            }catch (e){
                setTimeout(_flushCallback,0);
            }
        }
    };
    
    const requestHostCallback = function(cb):void{
        //当有正在执行的callback时，把cb加入到队列。
        if(_callback !== null){
            setTimeout(requestHostCallback,0,cb);
        }else {
        //当前没有执行函数时，执行cb
            _callback = cb;
            setTimeout(_flushCallback,0);
        }
    };
    
    const cancelHostCallback = function():void{
        _callback = null;
    };
    
    const requestHostTimeout = function(cb,ms:number):void{
        _timeoutID = setTimeout(cb,ms);
    };
    
    const cancelHostTimeout = function():void{
        clearTimeout(_timeoutID);
    };
    
    const shouleYieldToHost = function():boolean{
        return false;
    };
    
    const requestPaint = function(){};
    const forceFrameRate = function(){};
}

In [ ]:
{
    /*scheduleFeatureflags.js*/
    const enableIsInputPending = false;
    const enableMessageLoopImplementation = true;
    
    let requestHostCallback;
    let cancelHostCallback;
    let requestHostTimeout;
    let cancelHostTimeout;
    let shouldYieldToHost:boolean;
    let requestPaint;
    let getCurrentTime:number;
    let forceFrameRate;
    
    const performance = window.performance;
    const Date = window.Date;
    const setTimeout = window.setTimeout;
    const clearTimeout = window.clearTimeout;
    const requestAnimationFrame = window.requestAnimationFrame;
    const cancelAnimationFrame = window.cancelAnimationFrame;
    
    if(typeof performance === 'object' && typeof performance.now === 'function'){
        getCurrentTime = ()=> performance.now();
    }else{
        const initialTime = Date.now();
        getCurrentTime = ()=>Date.now() - initialTime;
    }
    
    let isRAFLoopRunning:boolean = false;
    let isMessageLoopRunning:boolean = false;
    let scheduledHostCallback = null;
    let rAFTimeoutID = -1;
    let taskTimeoutID = -1;
    
    let frameLength = enableMessageLoopImplementation?5:33.33;
    
    let prevRAFTime = -1;
    let prevRAFInterval = -1;
    let frameDeadline = 0;
    
    let fpsLocked = false;
    
    let maxFrameLength = 300;
    let needsPaint = false;
    
    if(
        enableIsInputPending &&
        navigator !== undefined &&
        navigator.scheduling !== undefined &&
        navigator.scheduling.isInputPending !== undefined
    ) {
        const scheduling = navigator.scheduling;
        shouldYieldToHost = function():boolean{
            const currentTime = getCurrentTime();
            if(currentTime >= frameDeadline ) {
                if(needsPaint || scheduling.isInputPendng()){
                    return true;
                }
                return currentTime >= frameDeadline + maxFrameLength;
            }else{
                return false;
            }
        };
        
        requestPaint = function(){
            needsPaint = true;
        }
    } else {
        shouldYieldToHost = function():boolean{
            return getCurrentTime() >= frameDeadline;
        };
        
        requestPaint =function(){}    
    }
    
    forceFrameRate = function(fps:number):void {
        if(fps < 0 || fps > 125){
            console.error(
                `forceFrameRate takes a positive int between 0 and 125, `+
                `forcing framerates higher than 125 fps is not unsupported.`
            );
            return;
        }
        if(fps>0){
            frameLength = Math.floor(1000/fps);
            fpsLocked = true;
        }else {
            frameLength = 33.33;
            fpsLocked = false;
        }
    }
    
    const performWorkUntilDeadline = ()=>{
        if(enableMessageLoopImplementation){
            if(scheduledHostCallback !== null){
                const currentTime = getCurrentTime();
                frameDeadline = currentTime + frameLength;
                const hasTimeRemaining = true;
                
                try{
                    const hasMoreWork = scheduledHostCallback(
                        hasTimeRemaining,
                        currentTime
                    );
                    if(!hasMoreWork){
                        isMessageLoopRunning = false;
                        scheduledHostCallback = null;
                    }else{
                        port.postMessage(null);
                    }
                }catch (error){
                    port.postMessage(null);
                    throw error;
                }
            }else {
                isMessageLoopRunning = false;
            }
            needsPaint = false;
        }else {
            if(scheduledHostCallback !== null){
                const currentTime = getCurrentTime();
                const hasTimeRemaining = frameDeadline - currentTime > 0;
                try {
                    const hasMoreWork = scheduledHostCallback(
                        hasTimeRemaining,
                        currentTime
                    );
                    if(!hasMoreWork){
                        scheduledHostCallback = null;
                    }
                }catch (error) {
                    port.postMessage(null);
                    throw error;
                }
            }
            needsPaint = false;
        }
    };
    
    const channel = new MessageChannel();
    const port = channel.port2;
    channel.port1.onmessage = performWorkUntilDeadline;
    
    const onAnimationFrame = rAFTime => {
        if(scheduledHostCallback === null){
            prevRAFTime = -1;
            prevRAFInterval = -1;
            isRAFLoopRunning = false;
            return;
        }
        
        isRAFLoopRunning = true;
        requestAnimationFrame(nextRAFTime => {
            clearTimeout(rAFTimeoutID);
            onAnimationFrame(nextRAFTime);
        })
        
        const onTimeout = ()=>{
            frameDeadline = getCurrentTime() + frameLength/2;
            performWorkUntilDeadline();
            rAFTimeoutID = setTimeout(onTimeout,frameLength*3);
        };
        rAFTimeoutID = setTimeout(onTimeout, frameLength*3);
        
        if(
            prevRAFTime !==-1 &&
            rAFTime - prevRAFTime > 0.1
        ){
            const rAFInterval = rAFTime - prevRAFTime;
            if(!fpsLocked && prevRAFInterval !== -1){
                if(rAFInterval < frameLength && prevRAFInterval < frameLength){
                    frameLength = rAFInterval < prevRAFInterval ? prevRAFInterval:rAFInterfal;
                    if(frameLength < 8.33){
                        frameLength = 8.33;
                    }
                }
            }
            prevRAFInterval = rAFInterval;
        }
        prevRAFTime = rAFTime;
        frameDeadline = rAFTime + frameLength;
        
        port.postMessage(null);
    }
    
    requestHostCallback = function(callback){
        scheduledHostCallback =callback;
        if(enableMessageLoopImplementation){
            if(!isMessageLoopRunning) {
                isMessageLoopRunning = true;
                port.postMessage(null);
            }
        }else {
            if(!isRAFLoopRunning) {
                isRAFLoopRunning = true;
                requestAnimationFrame(rAFTime => {
                    onAnimationFrame(rAFTime);
                })
            }
        }
    }
    
    cancelHostCallback = function(){
        scheduledHostCallback = null;
    }
    
    requestHostTimeout = function(callback,ms){
        taskTimeoutID = setTimeout(()=>{
            callback(getCurrentTime());
        },ms);
    };
    
    cancelHostTimeout = function(){
        clearTimeout(taskTimeoutID);
        taskTimeoutID = -1;
    }
    
}

流程图
[schedulerHostConfig.default.js 流程图](./schedulerHostConfig.default.dio)

## scheduleProfiling.js

记录日志。


In [ ]:
{
    let enableProfiling:boolean = true; //develpment下是true;product false;
    let NoPriority:number = 0;
    let runIdCounter:number = 0;
    let mainThreadIdCounter:number = 0;
    
    const profilingStateSize = 4;
    const shareProfillingBuffer = enableProfiling?
          typeof ShareArrayBuffer === 'function' ?
              new ShareArrayBuffer(profilingStateSize * Int32Array.BYTES_PER_ELEMENT):
              typeof ArrayBuffer === 'function'?
                  new ArrayBuffer(profilingStateSize * Int32Array.BYTES_PER_ELEMENT):
                  null
            :null;
    
    const profilingState = enableProfiling && shareProfilingBuffer!== null ?
          new Int32Array(shareProfilingBuffer):
          [];
    
    const PRIORITY = 0;
    const CURRENT_TASK_ID = 1;
    const CURRENT_RUN_ID = 2;
    const QUEUE_SIZE = 3;
    
    if(enableProfiling){
        profilingState[PRIORITY] = NoPriority;
        profilingState[QUEUE_SIZE] = 0;
        profilingState[CURRENT_TASK_ID] = 0;
    }
    
    //Bytes per eleent is 4
    const INITIAL_EVENT_LOG_SIZE = 131072;
    const MAX_EVENT_LOG_SIZE = 524288; //equivalent to 2 megabytes 4*INITIAL_EVNET_LOG_SIZE;
    
    let eventLogSize = 0;
    let eventLogBuffer = null;
    let eventLog = null;
    let eventLogIndex =0;
    
    const TaskStartEvent = 1;
    const TaskCompleteEvent = 2;
    const TaskErrorEvent = 3;
    const TaskCancelEvent = 4;
    const TaskRunEvent = 5;
    const TaskYieldEvent = 6;
    const SchedulerSuspendEvent = 7;
    const SchedulerResumeEvent = 8;
    
    function logEvent(entries) {
        if(eventLog !== null){
            const offset = eventLogIndex;
            eventLogIndex += entries.length;
            if(eventLogIndex+1 > eventLogSize){
                eventLogSize *= 2;
                if(eventLogSize > MAX_EVENT_LOG_SIZE){
                    console.error(
                        `scheduler profiling: event log exceeded maximum size. don't` +
                        `forget to call "stopLogginggProfilingEvents()" .`
                    );
                    stopLoggingProfilingEvents();
                    return;
                }
                const newEventLog = new Int32Array(eventLogSize * 4);
                newEventLog.set(eventLog);
                eventLogBuffer = newEventLog.buffer;
                eventLog = newEventLog;
            }
            eventLog.set(entries,offset);
        }
    }
    
    function startLoggingProfilingEvents():void {
        eventLogSize = INITIAL_EVENT_LOG_SIZE;
        eventLogBuffer = new ArrayBuffer(eventLogSize * 4);
        eventLog = new Int32Array(eventLogBuffer);
        eventLogIndex = 0;
    }
             
    function stopLoggingProfilingEvents():ArrayBuffer|null {
        const buffer = eventLogBuffer;
        eventLogSize = 0;
        eventLogBuffer = null;
        eventLog = null;
        eventLogIndex = 0;
        return buffer;
    }
    
    function markTaskStart(
        task:{id:number,prioityLevel:PriorityLevel},
         ms:number
    ){
        if(enableProfiling){
            profilingState[QUEUE_SIZE]++;
            
            if(eventLog !== null){
                logEvent([TaskStartEvent, ms*1000, task.id, task.priorityLevel]);
            }
        }
    }
    
    function markTaskCompleted(
        task:{ id:number, priorityLevel:PriorityLevel},
         ms:number
    ){
        if(enableProfiling){
            profilingState[PRIORITY] = NoPriority;
            profilingState[CURRENT_TASK_ID] = 0;
            profilingState[QUEUE_SIZE]--;
            
            if(eventLog!==null){
                logEvent([TaskCompleteEvent, ms*1000, task.id]);
            }
        }
    }
    
    function markTaskCanceled(
        task:{id:number,priorityLevel:PriorityLevel},
        ms:number
    ){
        if(enableProfiling){
            profilingState[QUEUE_SIZE]--;
            if(eventLog !== null){
                logEvent([TaskCancelEvent,ms*1000,task.id]);
            }
        }
    }
    
    function markTaskErrored(
        task:{id:number,priorityLevel:PriorityLevel},
        ms:number
    ){
        if(enableProfiling){
            profilingState[PRIORITY] = NoPriority;
            profilingState[CURRENT_TASK_ID] = 0;
            profilingState[QUEUE_SIZE]--;
            
            if(eventLog!==null){
                logEvent([TaskErrorEvent, ms*1000, task.id]);
            }
        }
    }
    
    function markTaskRun(
        task:{id:number,priorityLevel:PriorityLevel},
        ms:number
    ){
        if(enableProfiling){
            runIdCounter++;
            
            profilingState[PRIORITY] = task.priorityLevel;
            profilingState[CURRENT_TASK_ID] = 0;
            profilingState[CURRENT_RUN_ID] = 0;
            
            if(eventLog !== null){
                logEvent([TaskRunEvent, ms*1000, task.id, runIdCounter]);
            }
        }
    }
    
    function markTaskYield(task:{id:number},ms:number){
        if(enableProfiling){
            profilingState[PRIORITY] = NoPriority;
            profilingState[CURRENT_TASK_ID] = 10;
            profilingState[CURRENT_RUN_ID] = 0;
        }
        
        if(eventLog !== null){
            logEvent([TaskYieldEvent, ms*1000, task.id, runIdCounter]);
        }
    }
    
    function markSchedulerSuspended(ms:number){
        if(enableProfiling){
            mainThreadIdCounter++;
            
            if(eventLog!==null){
                logEvent([SchedulerSuspendEvent, ms*1000, mainThreadIdCounter]);
            }
        }
    }
    
    function markSchedulerUnsuspended(ms:number){
        if(enableProfiling){
            if(eventLog!==null){
                logEvent([SchedulerResumeEvent, ms* 1000, mainThreadIdCounter]);
            }
        }
    }
}

think list

- [ ] 类型化数组
    - [ ] ShareArrayBuffer
    - [ ] ArrayBuffer

## scheduler.js

In [ ]:
{
    let maxSigned31BitInt = 1073741823;
    let IMMEDIATE_PRIORITY_TIMEOUT = -1;
    let USER_BLOCKING_PRIORITY = 250;
    let NORMAL_PRIORITY_TIMEOUT = 5000;
    let LOW_PRIORITY_TIMEOUT = 10000;
    let IDLE_PRIORITY = maxSigned31BitInt;
    
    let taskQueue = [];//当前事件队列，应执行的队列事件
    let timerQueue =[];//当前不用执行的队列，延迟事件队列
    // taskQueue timerQueue 之间是通过 expirationTime starttune currenttime 几个参数来判断的
    // timerQueue 里的事件当expirationtime 超过当前时间则 立即取出 加入到taskQueue里
    // 当有新事件产生时 是通过starttime currenttime 来判断 是放到taskQueue里 还是timerQueue里
    
    let taskIdCounter = 1;
    let isSchedulerPaused = false;
    
    let currentTask = null;
    let currentPriorityLevel = NormalPriority;
    
    let isPerformingWork = false;
    
    let isHostCallbackScheduled = false;
    let isHostTimeoutScheduled = false;
    
    //清理时间队列中的事件 过期或者被取消
    
    //todo 不明白为何配到不是这两种情况的时候直接return 
    //难道后续的队列里 不会有这种情况？
    function advanceTimers(currentTime){
        let timer = peek(timeQueue);
        while(timer!==null){
            if(timer.callback===null){
                pop(timerQueue);
            }else if(timer.startTime <= currentTime){
                pop(timerQueue);
                timer.sortIndex = timer.expirationTime;
                push(taskQueue, timer);
                if(enableProfiling){
                    markTaskStart(timer,currentTime);
                    timer.isQueued = true;
                }
            }else {
                return ;
            }
            timer = peek(timerQueue);
        }
    }
    
    function handleTimeout(currentTime){
        isHostTimeoutScheduled = false;
        advanceTimers(currentTime);
        
        if(!isHostCallbackScheduled){
            isHostCallbackScheduled = true;
            requestHostCallback(flushWork);
        }else {
            const firstTimer = peek(timeQueue);
            if(firstTimer!== null){
                requestHostTimeout(handleTimeout, firstTimer.startTime - currentTime);
            }
        }
    }
    
    function flushWork(hasTimeRemaining, initialTime){
        if(enableProfiling){
            markSchedulerUnsuspened(initialTime);
        }
        
        isHostCallbackScheduled = false;
        if(isHostTimeoutScheduled){
            isHostTimeoutScheduled = false;
            cancelHostTimeout();
        }
        
        isPerformingWork = true;
        const previousPriorityLevel = currentPriorityLevel;
        try {
            if(enableProfiling){
                try {
                    return workLoop(hasTimeRemaining, initialTime);
                }catch(error){
                    if(currentTask !== null) {
                        const currentTime = getCurrentTime();
                        markTaskErrored(currentTask, currentTime);
                        currentTask.isQueued = false;
                    }
                    throw error;
                }
            }else{
                return workLoop(hasTimeRemaining, initialTime);
            }
        }finally {
            currentTask = null;
            currentPriorityLevel = previousPriorityLevel;
            isPerformingWrok = false;
            if(enableProfiling){
                const currentTime = getCurrentTime();
                markSchedulerSuspended(currentTime);
            }
        }
    }
    
    function workLoop(hasTimeRemainng,initialTime){
        let currentTime = initialTime;
        advanceTimers(currentTime);
        currentTask = peek(taskQueue);
        while(
            currentTask !== null &&
            !(enableSchedulerDebugging || isSchedulerPaused)
        ){
            if(
                currentTask.expirationTime > currentTime &&
                (!hasTimeRemaining || shouldYieldToHost())
            ){
                break;
            }
            const callback = currentTask.callback;
            if(callback!==null){
                currentTask.callback = null;
                currentPriorityLevel = currentTask.priorityLevel;
                const didUserCallbackTimeout = currentTask.expirationTime <= currentTime;
                markTaskRun(currentTask, currentTime);
                const continuationCallback = callback(didUserCallbackTimeout);
                currentTime = getCurrentTime();
                if(typeof continuationCallback === 'function'){
                    currentTask.callback = continuationCallback;
                    markTaskYield(currentTask, currentTime);
                }else{
                    if(enableProfiling){
                        markTaskCompleted(currentTask,currentTime);
                        currentTask.isQueued = false;
                    }
                    if(currentTask === peek(taskQueue)){
                        pop(taskQueue);
                    }
                }
                advanceTimers(currentTime);
            }else{
                pop(taskQueue);
            }
            currentTask = peek(taskQueue);
        }
        if(currentTask !== null){
            return true;
        }else {
            let firstTimer = peek(timerQueue);
            if(firstTimer!== null){
                requestHostTimeout(handleTimeout, firstTimer.startTime - currentTime);
            }
            return false;
        }
    }
    
    function unstable_runWithPriority(priorityLevel,eventHandler){
        switch(priorityLevel){
            case ImmediatePriority:
            case UserBlockingPriority:
            case NormalPriority:
            case LowPriority:
            case IdlePriority:
                break;
            default:
                priorityLevel = NormalPriority;
        }
        
        let previousPriorityLevel = currentPriorityLevel;
        currentPriorityLevel = priorityLevel;
        
        try {
            return eventHandler();
        }finally{
            currentPriorityLevel = previousPriorityLevel;
        }
    }
    
    function unstable_next(eventHandler){
        var priorityLevel;
        switch(currentPriorityLevel){
            case ImmediatePriority:
            case UserBlockingPriority:
            case NormalPriority:
                priorityLevel = NormalPriority;
                break;
            default:
                priorityLevel = currentPriorityLevel;
                break;
        }
        
        let previousPriorityLevel = currentPriorityLevel;
        currentPriorityLevel = priorityLevel;
        
        try{
            return eventHandler();
        }finally{
            currentPriorityLevel = previousPriorityLevel;
        }
    }
    
    function unstable_wrapCallback(callback){
        let parentPriorityLevel = currentPriorityLevel;
        return function(){
            let previousPriorityLevel = currentPriorityLevel;
            currentPriorityLevel = parentPriorityLevel;
            
            try{
                return callback.apply(this, arguments);
            }finally{
                currentPriorityLevel = previousPriorityLevel;
            }
        }
    }
    
    function timeoutForPriorityLevel(priorityLevel){
        switch(priorityLevel){
            case ImmediatePriority:
                return IMMEDIATE_PRIORITY_TIMEOUT;
            case UserBlockingPriority:
                return USER_BLOCKING_PRIORITY;
            case IdlePriority:
                return IDLE_PRIORITY;
            case LowPriority:
                return LOW_PRIORITY_TIMEOUT;
            case NormalPriority:
                return NORMAL_PRIORITY_TIMEOUT;
        }
    }
    
    
    //关键函数
    //新加入事件的逻辑判断
    //加入 taskQueue or timerQueue
    function unstable_scheduleCallback(priorityLevel, callback, options){
        let currentTime = getCurrentTime();
        
        let startTime;
        let timeout;
        if(typeof options === 'object' && options !== null){
            let delay = options.delay;
            if(typeof delay === 'number' && delay > 0){
                startTime = currentTime + delay;
            }else{
                startTime = currentTime;
            }
            
            timeout = typeof options.timeout === 'number'?
                    options.timeout:
                    timeoutForPriorityLevel(priorityLevel);
        }else{
            timeout = timeoutForPriorityLevel(priorityLevel);
            startTime = currentTime;
        }
        
        let expirationTime = startTime + timeout;
        
        let newTask = {
            id:taskIdCounter++,
            callback,
            priorityLevel,
            startTime,
            expirationTime,
            sortIndex:-1
        };
        
        if(enableProfiling){
            enwTask.isQueued = false;
        }
        
        if(startTime > currentTime){
            newTask.sortIndex = startTime;
            push(timerQueue, newTask);
            if(peek(taskQueue)===null && newTask === peek(timerQueue)){
                if(isHostTimeoutScheduled){
                    cancelHostTmeout();
                }else{
                    isHostTimeoutScheduled =true;
                }
                
                requestHostTimeout(handleTimeout, startTime - currentTime);
            }
        }else{
            newTask.sortIndex = expirationTime;
            push(tashQueue, newTask);
            if(enableProfiling){
                markTaskStart(newTask,currentTime);
                newTask.isQueued = true;
            }
            
            if(!isHostCallbackScheduled && !isPerformingWork){
                isHostCallbackScheduled = true;
                requestHostCallback(flushWork);
            }
        }
        return newTask;
    }
    
    function unstable_pauseExecution(){
        isSchedulerPaused = true;
    }
    
    function unstable_continueExecution(){
        isSchedulerPaused = false;
        if(!isHostCallbackScheduled && !isPerformingWork){
            isHostCallbackScheduled = true;
            requestHostCallback(flushWork);
        }
    }
    
    function unstable_getFirstCallbackNode(){
        return peek(taskQueue);
    }
    
    function unstable_cancelCallback(task){
        if(enableProfiling){
            if(task.isQueued){
                const currentTime = getCurrentTime();
                markTaskCanceled(task,currentTime);
                task.isQueued = false;
            }
        }
        /*
        Null out the callback to indicate the task has been canceled. (can't remove fro the queue
        becase you can't remove arbitrary nodes from an array based heap, only the first)
        */
        task.callback = null;
    }
    
    function unstable_getCurrentPriorityLevel(){
        return currentPriorityLevel;
    }
    
    function unstable_shouldYield(){
        const currentTime = getCurrentTime();
        advanceTimers(currentTime);
        const firstTask = peek(taskQueue);
        return (
            (
                firstTask !== currentTask &&
                currentTask !== null &&
                firstTask !== null &&
                firstTask.callback !== null &&
                firstTask.startTime <= currentTime &&
                firstTask.expirationTime < currentTask.expirationTime
            ) || shouldYieldToHost()
        );
    }
    
    const unstable_requestPaint = requestPaint;
    
    export {
      ImmediatePriority as unstable_ImmediatePriority,
      UserBlockingPriority as unstable_UserBlockingPriority,
      NormalPriority as unstable_NormalPriority,
      IdlePriority as unstable_IdlePriority,
      LowPriority as unstable_LowPriority,
      unstable_runWithPriority,
      unstable_next,
      unstable_scheduleCallback,
      unstable_cancelCallback,
      unstable_wrapCallback,
      unstable_getCurrentPriorityLevel,
      unstable_shouldYield,
      unstable_requestPaint,
      unstable_continueExecution,
      unstable_pauseExecution,
      unstable_getFirstCallbackNode,
      getCurrentTime as unstable_now,
      forceFrameRate as unstable_forceFrameRate,
    };
    
    export const unstable_profiling = enableProfiling ? {
        startLoggingProfilingEvents,
        stopLoggingProfilingEvents,
        shareProfilingBuffer
    }:null;
}

scheduler

简单来说就是维护两个事件队列

一个主队列 taskQueue  一个延迟队列timerQueue

每次事件执行时都会去确认下 timerQueue 下是否有事件取消或者要加入到taskQueue里

workLoop 是整个事件循环

taskQueue 是一个最小堆，也就意味着加入时，会根据优先级重新整理堆。获取的第一个永远是优先级最高的。同理timerQueue一样
取消未执行的taskQueue里的事件，具体操作就是把task 的callback 置为 null。